In [1]:
%pip install -q flwr[simulation] 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires ptxcompiler, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires protobuf<4,>3.12.2, but you have protobuf 4.25.3 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 15.0.2 which is incompatible.
cudf 23.8.0 requires cuda

In [2]:
import numpy as np
import os
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import flwr as fl
import matplotlib.pyplot as plt
import ipywidgets as widgets
import IPython.display as display_output
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import random_split, DataLoader
from collections import OrderedDict
from typing import List, Tuple
from IPython.display import display, clear_output
from IPython.display import display, HTML, clear_output
from flwr.common import Metrics
from flwr.server.strategy import FedAdam
global num_c, trainloaders, valloaders, testloader, model, classes, output
num_clients=15

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"  # Required for Kaggle to work with PyTorch

torch.cuda.is_available()  # Check if GPU is available
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(
    f"Training on {DEVICE} using PyTorch {torch.__version__}"
)
# Dataset paths
dataset_paths = '/kaggle/input/pneumonia/chest_xray'

datatrain = f'{dataset_paths}/train'
datatest = f'{dataset_paths}/test'

BATCH_SIZE = 32
img_size = (224, 224)

# Download and transform
transform = transforms.Compose([
    transforms.Grayscale(),
    transforms.Resize(img_size),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

trainset = ImageFolder(datatrain, transform=transform)
testset = ImageFolder(datatest, transform=transform)
size_label = widgets.Label()
size_label.value = f'The size of data: {len(trainset) + len(testset)}'
print(size_label)

# Get the classes from the trainset
classes = trainset.classes
num_c = len(classes)

# Split training set into num_clients partitions to simulate the individual dataset
partition_size = len(trainset) // num_clients
remainder = len(trainset) % num_clients
lengths = [partition_size + remainder] + [partition_size] * (num_clients - 1)
datasets = random_split(trainset, lengths, torch.Generator().manual_seed(42))

# Split each partition into train/val and create DataLoader
trainloaders = []
valloaders = []
for ds in datasets:
    len_val = len(ds) // 10  # 10 % validation set
    len_train = len(ds) - len_val
    lengths = [len_train, len_val]
    ds_train, ds_val = random_split(ds, lengths, torch.Generator().manual_seed(42))
    trainloaders.append(DataLoader(ds_train, batch_size=BATCH_SIZE, shuffle=True))
    print("hi")
    valloaders.append(DataLoader(ds_val, batch_size=BATCH_SIZE))
testloader = DataLoader(testset, batch_size=BATCH_SIZE)
    

def get_parameters(net) -> List[np.ndarray]:
    return [val.cpu().numpy() for _, val in net.state_dict().items()]


def set_parameters(net, parameters: List[np.ndarray]):
    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)
    
class Net(nn.Module):
    def __init__(self, num_c):
        super(Net, self).__init__()
        self = self.to(DEVICE)
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.pool3 = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.5)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * 28 * 28, 264)
        self.fc2 = nn.Linear(264,num_c)

    def forward(self, x):
        x = self.conv1(x)
        x = nn.ReLU()(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = nn.ReLU()(x)
        x = self.pool2(x)
        x = self.conv3(x)
        x = nn.ReLU()(x)
        x = self.pool3(x)
        x = self.dropout(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = nn.ReLU()(x)
        x = self.fc2(x)
        return x
    
def train(net, trainloader, valloader, epochs: int,patience: int = 5, verbose=True):
    """Train the network on the training set."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters())
    net.train()
    best_val_acc = 0.0
    early_stop_counter = 0
    for epoch in range(epochs):
        correct, total, epoch_loss = 0, 0, 0.0
        for images, labels in trainloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            outputs = net(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            # Metrics
            epoch_loss += loss
            total += labels.size(0)
            correct += (torch.max(outputs.data, 1)[1] == labels).sum().item()
        epoch_loss /= len(trainloader.dataset)
        epoch_acc = correct / total
        val_loss, val_acc = test(net, valloader)

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            early_stop_counter = 0
        else:
            early_stop_counter += 1
            if early_stop_counter >= patience:
                print(f"Early stopping after {epoch + 1} epochs.")
                break

        if verbose:
            print(f"Epoch {epoch+1}: train loss {epoch_loss}, accuracy {epoch_acc}, val loss {val_loss}, val accuracy {val_acc}")
def test(net, testloader):
    """Evaluate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct, total, loss = 0, 0, 0.0
    net.eval()
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy

 
def aggregation_FedAdam(metrics: List[Tuple[int, dict]]) -> dict:
    # Multiply accuracy of each client by number of examples used
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]

    # Aggregate and return custom metric (weighted average)
    total_examples = sum(examples)
    weighted_acc_sum = sum(accuracies)

    # Apply FedAdam strategy (e.g., update the weighted average with global update)
    global_update = 0.1  # Replace with the appropriate global update value

    weighted_acc_sum += global_update * total_examples
    total_examples += global_update

    return {"accuracy": weighted_acc_sum / total_examples}

def fit_metrics_aggregation_fn(fit_metrics):
    # Aggregate and return custom metric (e.g., weighted average)
    accuracies = [num_examples * m["accuracy"] for num_examples, m in fit_metrics]
    examples = [num_examples for num_examples, _ in fit_metrics]
    return {"accuracy": sum(accuracies) / sum(examples)}

class FlowerNumPyClient(fl.client.NumPyClient):
    def __init__(self, cid, net, trainloader, valloader):
        self.cid = cid
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader
    def get_parameters(self, config):
        print(f"[Client {self.cid}] get_parameters")
        return get_parameters(self.net)
    def fit(self, parameters, config):
        print(f"[Client {self.cid}] fit, config: {config}")
        set_parameters(self.net, parameters)
        train(self.net, self.trainloader,self.valloader, epochs=1)
        
        # Get the accuracy on the training set
        correct, total = 0, 0
        self.net.eval()
        with torch.no_grad():
            for images, labels in self.trainloader:
                images, labels = images.to(DEVICE), labels.to(DEVICE)
                outputs = self.net(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        accuracy = correct / total
        
        return get_parameters(self.net), len(self.trainloader), {"accuracy": float(accuracy)}
    def evaluate(self, parameters, config):
        print(f"[Client {self.cid}] evaluate, config: {config}")
        set_parameters(self.net, parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}
    
    
def client_fn(cid) -> FlowerNumPyClient:
    net = Net(num_c).to(DEVICE)
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]
    flower_numpy_client = FlowerNumPyClient(cid, net, trainloader, valloader)
    return flower_numpy_client.to_client()

def run_simulation(num_round,strategy_dropdown):   
    if strategy_dropdown == "FedAdam":
        # Start simulation with FedAdam strategy
        strategy = FedAdam(
            fraction_fit=1.0,
            fraction_evaluate=0.5,
            min_fit_clients=num_clients - 1,
            min_evaluate_clients=num_clients - 2,
            min_available_clients=num_clients,
            evaluate_metrics_aggregation_fn=aggregation_FedAdam,
            fit_metrics_aggregation_fn=fit_metrics_aggregation_fn,
            initial_parameters=fl.common.ndarrays_to_parameters(get_parameters(Net(num_c))),
            eta=0.001,  # Adjust the learning rate if needed
            eta_l=1e-1,  # Adjust the client-side learning rate if needed
            beta_1=0.9,  # Adjust the momentum parameter if needed
            beta_2=0.99,  # Adjust the second moment parameter if needed
            tau=1e-9,
        )

    # Update with your specific simulation code
    fl.common.logger.configure(identifier="myFlowerExperiment", filename="log.txt")
    output = fl.simulation.start_simulation(
        client_fn=client_fn,
        num_clients=num_clients,
        config=fl.server.ServerConfig(num_rounds=num_round),
        strategy=strategy,
        client_resources={"num_cpus": 1, "num_gpus": 1},
    )

run_simulation(15,"FedAdam")

2024-05-08 06:22:35.240762: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-08 06:22:35.240885: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-08 06:22:35.339463: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-08 06:22:43,674	INFO util.py:129 -- Outdated packages:
  ipywidgets==7.7.1 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


Training on cuda using PyTorch 2.1.2


/opt/conda/lib/python3.10/site-packages/ipywidgets/widgets/widget.py:503: DeprecationWarning: The `ipykernel.comm.Comm` class has been deprecated. Please use the `comm` module instead.For creating comms, use the function `from comm import create_comm`.
  self.comm = Comm(**args)


Label(value='The size of data: 5840')
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi
hi


INFO :      Starting Flower simulation, config: num_rounds=15, no round_timeout
2024-05-08 06:22:54,471	INFO worker.py:1621 -- Started a local Ray instance.
INFO :      Flower VCE: Ray initialized with resources: {'object_store_memory': 8880961536.0, 'CPU': 4.0, 'node:__internal_head__': 1.0, 'node:172.19.2.2': 1.0, 'memory': 17761923072.0, 'GPU': 2.0}
INFO :      Optimize your simulation with Flower VCE: https://flower.ai/docs/framework/how-to-run-simulations.html
INFO :      Flower VCE: Resources for each Virtual Client: {'num_cpus': 1, 'num_gpus': 1}
INFO :      Flower VCE: Creating VirtualClientEngineActorPool with 2 actors
INFO :      [INIT]
INFO :      Using initial global parameters provided by strategy
INFO :      Evaluating initial global parameters
INFO :      
INFO :      [ROUND 1]
INFO :      configure_fit: strategy sampled 15 clients (out of 15)
(pid=301) 2024-05-08 06:22:58.621496: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN fa

...

INFO :      aggregate_evaluate: received 13 results and 0 failures
INFO :      
INFO :      [ROUND 15]
INFO :      configure_fit: strategy sampled 15 clients (out of 15)


(ClientAppActor pid=301) [Client 4] fit, config: {}
(ClientAppActor pid=301) Epoch 1: train loss 0.009128458797931671, accuracy 0.7699680511182109, val loss 0.12542596284080954, val accuracy 0.7352941176470589
(ClientAppActor pid=301) [Client 6] evaluate, config: {} [repeated 2x across cluster]
(ClientAppActor pid=300) [Client 11] fit, config: {}
(ClientAppActor pid=301) [Client 9] fit, config: {}
(ClientAppActor pid=300) Epoch 1: train loss 0.011351363733410835, accuracy 0.7507987220447284, val loss 0.016256711938801932, val accuracy 0.8235294117647058
(ClientAppActor pid=301) Epoch 1: train loss 0.013270405121147633, accuracy 0.7571884984025559, val loss 0.03504483840044807, val accuracy 0.7647058823529411
(ClientAppActor pid=300) [Client 12] fit, config: {}
(ClientAppActor pid=301) [Client 13] fit, config: {}
(ClientAppActor pid=300) Epoch 1: train loss 0.011432847008109093, accuracy 0.8019169329073482, val loss 0.01937347650527954, val accuracy 0.8235294117647058
(ClientAppActor pi

INFO :      aggregate_fit: received 15 results and 0 failures
INFO :      configure_evaluate: strategy sampled 13 clients (out of 15)


(ClientAppActor pid=300) [Client 9] evaluate, config: {}
(ClientAppActor pid=300) [Client 2] evaluate, config: {} [repeated 10x across cluster]


INFO :      aggregate_evaluate: received 13 results and 0 failures
INFO :      
INFO :      [SUMMARY]
INFO :      Run finished 15 rounds in 1560.21s
INFO :      History (loss, distributed):
INFO :      	('\tround 1: 0.03953443636182207\n'
INFO :      	 '\tround 2: 0.038050322445577864\n'
INFO :      	 '\tround 3: 0.03599770018263794\n'
INFO :      	 '\tround 4: 0.03249386992081566\n'
INFO :      	 '\tround 5: 0.02777569284825149\n'
INFO :      	 '\tround 6: 0.021315870754319618\n'
INFO :      	 '\tround 7: 0.020374457475999593\n'
INFO :      	 '\tround 8: 0.020095675805151345\n'
INFO :      	 '\tround 9: 0.021622174432518956\n'
INFO :      	 '\tround 10: 0.024259740893374442\n'
INFO :      	 '\tround 11: 0.025770820405521762\n'
INFO :      	 '\tround 12: 0.01799085184502215\n'
INFO :      	 '\tround 13: 0.025365330637586146\n'
INFO :      	 '\tround 14: 0.023523515566151638\n'
INFO :      	 '\tround 15: 0.021671232023338392\n')History (metrics, distributed, fit):
INFO :      	{'accurac